## Setup

In [1]:
import sys, time
sys.path.insert(0, "../src/")
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
from matplotlib import pyplot as plt
import torch
from chexnet_medical_ai.image_preprocessing import ImageTransForm, ImageDataLoader
from chexnet_medical_ai import model

## Train and Test Loaders 

In [3]:
train_path = "/mnt/d/archive/train/"
test_path = "/mnt/d/archive/test/"

In [4]:
train_transform = ImageTransForm().train_image_transforms()
test_transform = ImageTransForm().test_image_transforms()

In [5]:
train_data = ImageDataLoader(train_path, test_path).train_data(train_transform)
test_data = ImageDataLoader(train_path, test_path).test_data(test_transform)

In [6]:
train_data.classes, test_data.classes

(['0', '1'], ['0', '1'])

In [7]:
torch.manual_seed(42)
train_loader = ImageDataLoader(train_path, test_path).train_data_loader(train_transform)
test_loader = ImageDataLoader(train_path, test_path).test_data_loader(test_transform)

## Model

In [8]:
cnn_model = model.ConvolutionalNewtwork()
cnn_model

ConvolutionalNewtwork(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=46656, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [ ]:
start_time = time.time()

epochs = 50
max_train_batch = 800
max_text_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    


total_time = time.time()
print(f"Time taken took for training (in minutes):- {total_time/60}")